In [1]:
%reload_ext autoreload
%autoreload 2
import numpy as np, sys, os, pandas as pd,json, os
sys.path.insert(1, '../../')
from getting_data import load_sample, feature_key_list, categorical_feature_key_list
from ranker_helper import get_scores
from s2search_score_pdp import save_pdp_to_npz

setting = 0
exp_name = 'exp5'
sample_name = 'cslg'

query = 'Machine Learning'

categorical_name = {}

def remove_duplicate(seq):
    seen = set()
    seen_add = seen.add
    return [x for x in seq if not (x in seen or seen_add(x))]

categorical_name_file = os.path.join('.', 'scores', f'{sample_name}_categorical_name.npz')

for i in range(len(feature_key_list)):
    feature_name = feature_key_list[i]
    if feature_name in categorical_feature_key_list:
        df = load_sample(exp_name, sample_name, query=query, sort=feature_name, rank_f=get_scores)
        if feature_name == 'authors':
            l = [json.dumps(x) for x in df[feature_name]]
        else:
            l = list(df[feature_name])
        categorical_name[i] = remove_duplicate(l)

fail to not force global because 1 worker available
get ranker in 77418 with global setting: True and gb_ranker len 1
[Main taks:-1] compute 92938 scores with worker 77418
[Main taks][03/26/2022, 13:51:04] 92938 scores within 35.166157 sec 
fail to not force global because 1 worker available
get ranker in 77418 with global setting: True and gb_ranker len 1
[Main taks:-1] compute 92938 scores with worker 77418
[Main taks][03/26/2022, 13:51:57] 92938 scores within 47.045227 sec 
fail to not force global because 1 worker available
get ranker in 77418 with global setting: True and gb_ranker len 1
[Main taks:-1] compute 92938 scores with worker 77418
[Main taks][03/26/2022, 13:52:18] 92938 scores within 15.226854 sec 
fail to not force global because 1 worker available
get ranker in 77418 with global setting: True and gb_ranker len 1
[Main taks:-1] compute 92938 scores with worker 77418
[Main taks][03/26/2022, 13:52:48] 92938 scores within 25.429801 sec 


In [2]:
df = load_sample(exp_name, sample_name)
paper_data = json.loads(df.to_json(orient='records'))

y_pred_file = os.path.join('.', 'scores', f'{sample_name}_y_pred.npz')
if not os.path.exists(y_pred_file):
    y_pred = get_scores(query, paper_data)
    save_pdp_to_npz('.', y_pred_file, y_pred)
else:
    y_pred = np.load(y_pred_file)['arr_0']

In [3]:
def get_class(score):
    if score <= -17:
        return '(,-17]'
    elif score <= -10:
        return '(-17, -10]'
    elif score <= -5:
        return '(-10, -5]'    
    elif score <= 0:
        return '(-5, <0]'  
    elif score <= 3:
        return '(0, 3]'
    elif score <= 5:
        return '(3, 5]'
    elif score <= 6:
        return '(5, 6]'
    elif score <= 7:
        return '(6, 7]'
    elif score <= 8:
        return '(7, 8]'
    elif score <= 9:
        return '(8, 9]'
    else:
        return '(9,)'

# make class_name
class_name = ['(,-17]','(-17, -10]','(-10, -5]','(-5, <0]','(0, 3]','(3, 5]','(5, 6]','(6, 7]','(7, 8]','(8, 9]','(9,)']

categorical_name_map = {}

for i in range(len(feature_key_list)):
    feature_name = feature_key_list[i]
    if feature_name in categorical_feature_key_list:
        categorical_name_map[i] = {}
        values = categorical_name[i]
        for j in range(len(values)):
            value = values[j]
            categorical_name_map[i][value] = j

# encoding data
for i in range(len(paper_data)):
    paper_data[i] = [
        categorical_name_map[0][paper_data[i]['title']], categorical_name_map[1][paper_data[i]['abstract']],
        categorical_name_map[2][paper_data[i]['venue']], categorical_name_map[3][json.dumps(paper_data[i]['authors'])],
        paper_data[i]['year'],
        paper_data[i]['n_citations']
    ]

In [10]:
def decode_paper(encoded_p):
    return dict(
        title=categorical_name[0][encoded_p[0]],
        abstract=categorical_name[1][encoded_p[1]],
        venue=categorical_name[2][encoded_p[2]],
        authors=json.loads(categorical_name[3][encoded_p[3]]),
        year=encoded_p[4],
        n_citations=encoded_p[5],
    )

decoded_i = 92937

print(paper_data[decoded_i])
print(df.iloc[decoded_i])
print(decode_paper(paper_data[decoded_i]))

[55397 28139  2995 84492  2019    61]
title          Pricing options and computing implied volatili...
abstract       This paper proposes a data-driven approach, by...
venue                                                      Risks
authors        [Shuaiqiang  Liu, Cornelis W. Oosterlee, Sande...
year                                                        2019
n_citations                                                   61
Name: 92937, dtype: object
{'title': 'Pricing options and computing implied volatilities using neural networks', 'abstract': 'This paper proposes a data-driven approach, by means of an Artificial Neural Network (ANN), to value financial options and to calculate implied volatilities with the aim of accelerating the corresponding numerical methods. With ANNs being universal function approximators, this method trains an optimized ANN on a data set generated by a sophisticated financial model, and runs the trained ANN as an agent of the original solver in a fast and eff

In [11]:
from anchor import anchor_tabular

paper_data = np.array(paper_data)

explainer = anchor_tabular.AnchorTabularExplainer(
    class_name,
    feature_key_list,
    paper_data,
    categorical_name)

In [59]:
def pred_fn(x):
    predictions = get_scores(query, [decode_paper(p) for p in x], ptf = False)
    encoded_pred = [class_name.index(get_class(pp)) for pp in predictions]
    return np.array(encoded_pred)

idx = 3
exp = explainer.explain_instance(paper_data[idx], pred_fn, threshold=0.9999, tau=0.6)

In [60]:
print('Prediction: ', explainer.class_names[pred_fn([paper_data[idx]])[0]])
print('-------------------')
print('Anchor: \n%s' % ('\nAND\n'.join(exp.names())))
print('\nPrecision: %.2f' % exp.precision())
print('Coverage: %.2f' % exp.coverage())

Prediction:  (-10, -5]
-------------------
Anchor: 
abstract = Lung cancer is the leading cause of cancer-related death worldwide. Early diagnosis of pulmonary nodules in Computed Tomography (CT) chest scans provides an opportunity for designing effective treatment and making financial and care plans. In this paper, we consider the problem of diagnostic classification between benign and malignant lung nodules in CT images, which aims to learn a direct mapping from 3D images to class labels. To achieve this goal, four two-pathway Convolutional Neural Networks (CNN) are proposed, including a basic 3D CNN, a novel multi-output network, a 3D DenseNet, and an augmented 3D DenseNet with multi-outputs. These four networks are evaluated on the public LIDC-IDRI dataset and outperform most existing methods. In particular, the 3D multi-output DenseNet (MoDenseNet) achieves the state-of-the-art classification accuracy on the task of end-to-end lung nodule diagnosis. In addition, the networks pretr

In [61]:
def extract_feature_keys(l):
    if len(l) == 0:
        return []
    return [anchor.split(' ')[0] for anchor in l]

previous_single_precision = 0
for i in range(len(exp.names())):
    previous_names = extract_feature_keys(exp.names(i - 1) if i > 0 else [])
    current_names = extract_feature_keys(exp.names(i))
    print([name for name in current_names if name not in previous_names][0], exp.precision(i) - previous_single_precision)
    previous_single_precision = exp.precision(i)

abstract 0.6831683168316832
title 0.2574257425742573
year 0.04378094059405946
n_citations -0.0062728102189780754
venue 0.021897810218978075


In [62]:
metrics = dict(
    title=[],
    abstract=[],
    venue=[],
    authors=[],
    year=[],
    n_citations=[],
)
previous_single_precision = 0

for j in range(len(exp.names())):
    name = exp.names()[j]
    print(name)
    current_single_precision = exp.precision(j) - previous_single_precision
    previous_single_precision = exp.precision(j)
    for feature_name in metrics.keys():
        if name.startswith(f'{feature_name}'):
            metrics[feature_name].append(current_single_precision)

metrics

abstract = Lung cancer is the leading cause of cancer-related death worldwide. Early diagnosis of pulmonary nodules in Computed Tomography (CT) chest scans provides an opportunity for designing effective treatment and making financial and care plans. In this paper, we consider the problem of diagnostic classification between benign and malignant lung nodules in CT images, which aims to learn a direct mapping from 3D images to class labels. To achieve this goal, four two-pathway Convolutional Neural Networks (CNN) are proposed, including a basic 3D CNN, a novel multi-output network, a 3D DenseNet, and an augmented 3D DenseNet with multi-outputs. These four networks are evaluated on the public LIDC-IDRI dataset and outperform most existing methods. In particular, the 3D multi-output DenseNet (MoDenseNet) achieves the state-of-the-art classification accuracy on the task of end-to-end lung nodule diagnosis. In addition, the networks pretrained on the LIDC-IDRI dataset can be further extend

{'title': [0.2574257425742573],
 'abstract': [0.6831683168316832],
 'venue': [0.021897810218978075],
 'authors': [],
 'year': [0.04378094059405946],
 'n_citations': [-0.0062728102189780754]}